In [3]:
!pip install urllib2

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


In [4]:
import json
import pandas as pd 

f = open('train/day-0.json')
data = json.load(f)
data

{'ovens': [{'start_temp': 1220,
   'working_temps': [960,
    1030,
    1190,
    1100,
    1230,
    1070,
    1110,
    1080,
    1240,
    1220],
   'operations': ['prokat', 'kovka']},
  {'start_temp': 990,
   'working_temps': [980, 1190, 1030, 1100, 990],
   'operations': ['otzhig', 'kovka']},
  {'start_temp': 1220,
   'working_temps': [970, 1060, 1220],
   'operations': ['otzhig']},
  {'start_temp': 1020,
   'working_temps': [1060, 1100, 1140, 1240, 1150, 1020],
   'operations': ['kovka']},
  {'start_temp': 1020,
   'working_temps': [970, 1100, 1070, 980, 1180, 990, 1020],
   'operations': ['otzhig']},
  {'start_temp': 1140,
   'working_temps': [1060, 1200, 1170, 1210, 1180, 1150, 1140],
   'operations': ['kovka']},
  {'start_temp': 1200,
   'working_temps': [960, 1100, 1190, 1030, 1200],
   'operations': ['otzhig']},
  {'start_temp': 1050,
   'working_temps': [960, 970, 1070, 1210, 1150, 1050],
   'operations': ['otzhig']},
  {'start_temp': 1020,
   'working_temps': [960, 970, 10

In [9]:
df = pd.json_normalize(data)
df

,ovens,series
0,"[{'start_temp': 1220, 'working_temps': [960, 1...","[{'temperature': 1080, 'operations': [{'name':..."


In [10]:
ovens = pd.json_normalize(data['ovens'])
ovens

,start_temp,working_temps,operations
0,1220,"[960, 1030, 1190, 1100, 1230, 1070, 1110, 1080...","[prokat, kovka]"
1,990,"[980, 1190, 1030, 1100, 990]","[otzhig, kovka]"
2,1220,"[970, 1060, 1220]",[otzhig]
3,1020,"[1060, 1100, 1140, 1240, 1150, 1020]",[kovka]
4,1020,"[970, 1100, 1070, 980, 1180, 990, 1020]",[otzhig]
...,...,...,...
417,1240,"[960, 1040, 1180, 1070, 1240]",[kovka]
418,1050,"[1030, 1160, 1040, 980, 1110, 1020, 990, 1050]",[prokat]
419,1140,"[1060, 1230, 1040, 1170, 990, 1140]",[kovka]
420,950,"[1030, 1190, 1160, 1070, 980, 950]",[prokat]


In [ ]:
# TODO: 
ovens_to_dumm

In [24]:
all_x = set()
all_y = set()

def do_uniq_f(x, y):
    x = tuple(x)
    y = tuple(y)
    if x not in all_x or y not in all_y:
        all_x.add(x)
        all_y.add(y)
        return 1
    
    return 0

ovens_uniq = [do_uniq_f(x, y) for x, y in zip(ovens['working_temps'], ovens['operations'])]
ovens_uniq

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [26]:
ovens_uniq.count(1)
ovens_uniq.count(0)

9

In [30]:
sorted(all_x)

[(960, 970, 980, 1110, 1240, 1210, 1230),
 (960, 970, 1070, 1010, 1170, 1020, 990),
 (960, 970, 1070, 1210, 1150, 1050),
 (960, 1000, 970, 1070, 1040, 1010, 990, 1150, 1100),
 (960, 1000, 970, 1100, 1200, 1240, 1050, 1080),
 (960, 1000, 1200, 980, 1080, 1210, 1180, 1190),
 (960, 1000, 1200, 1010, 1240, 1090),
 (960, 1000, 1230, 1200, 1180, 1170, 1110, 1080, 1020, 1140),
 (960, 1010, 1180, 1150, 1130),
 (960, 1030, 1160, 1100, 1150, 1240),
 (960, 1030, 1190, 1100, 1230, 1070, 1110, 1080, 1240, 1220),
 (960, 1040, 1180, 1070, 1240),
 (960, 1050, 1240, 1130),
 (960, 1060, 1030, 1000, 1100, 1040, 1200, 980, 1210, 1130),
 (960, 1060, 1030, 1160, 1200, 1010, 1140, 950, 1080, 990),
 (960, 1060, 1130, 970, 990, 950, 1050, 1180, 1150, 1080),
 (960, 1060, 1130, 1200, 950, 1240, 1210, 1010),
 (960, 1060, 1130, 1200, 1140, 1240, 1150),
 (960, 1060, 1220, 1160, 1130, 1110, 1150, 970),
 (960, 1070, 1230, 1170, 1240, 1150, 1130),
 (960, 1080, 1130, 980, 1240, 1210, 1160),
 (960, 1090, 970, 1020, 1080

In [11]:
series = pd.json_normalize(data['series'])
series

,temperature,operations
0,1080,"[{'name': 'nagrev', 'timing': 245}, {'name': '..."
1,970,"[{'name': 'nagrev', 'timing': 62}, {'name': 'o..."
2,1080,"[{'name': 'nagrev', 'timing': 76}, {'name': 'k..."
3,980,"[{'name': 'nagrev', 'timing': 96}, {'name': 'o..."
4,980,"[{'name': 'nagrev', 'timing': 82}, {'name': 'p..."
...,...,...
2589,960,"[{'name': 'nagrev', 'timing': 281}, {'name': '..."
2590,1060,"[{'name': 'nagrev', 'timing': 248}, {'name': '..."
2591,1000,"[{'name': 'nagrev', 'timing': 182}, {'name': '..."
2592,1090,"[{'name': 'nagrev', 'timing': 204}, {'name': '..."


In [14]:
print(series.iloc[0])

temperature                                                 1080
operations     [{'name': 'nagrev', 'timing': 245}, {'name': '...
Name: 0, dtype: object


In [50]:
# excel 
series = pd.read_excel('ПланОМД.xlsm', sheet_name='Серии')
series

,статус,Сплав,Гр. спл.,№слитка-индекс-часть,"кол. в садке, шт",Бойки,Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,NaN,6AL-4V,Ti,"34-8993-18№1,2",2.0,2.0,950.0,115-105,105-90,90-90,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,NaN,6AL-4V,Ti,"34-8993-18№3,4",2.0,2.0,950.0,115-105,105-90,90-90,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,NaN,6AL-4V,Ti,"34-8993-18№5,6",2.0,2.0,950.0,115-105,105-90,90-90,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
344,NaN,ВТ3-1,Ti,2513-18,2.0,NaN,960.0,220-150,NaN,NaN,NaN,NaN
345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
346,NaN,ВТ3-1,Ti,2523-18,2.0,NaN,960.0,220-150,NaN,NaN,NaN,NaN


In [51]:
series = series.drop(columns=['статус', 'Сплав', 'Гр. спл.', 'Бойки'])
series

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,"34-8993-18№1,2",2.0,950.0,115-105,105-90,90-90,NaN,NaN
1,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,"34-8993-18№3,4",2.0,950.0,115-105,105-90,90-90,NaN,NaN
3,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,"34-8993-18№5,6",2.0,950.0,115-105,105-90,90-90,NaN,NaN
...,...,...,...,...,...,...,...,...
343,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
344,2513-18,2.0,960.0,220-150,NaN,NaN,NaN,NaN
345,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
346,2523-18,2.0,960.0,220-150,NaN,NaN,NaN,NaN


In [52]:
series.iloc[0, 3:8]
# for i in range(0, 10, 2):
#     print(i)
# series.drop(labels = [1],axis = 0)

нагрев        115-105
подогрев       105-90
подогрев.1      90-90
подогрев.2        NaN
подогрев.3        NaN
Name: 0, dtype: object

In [53]:
for i in range(0, series.shape[0], 2):
    series.iloc[i, 3:8] = series.iloc[i+1, 3:8]
    # series = series.drop(labels = [i+1],axis = 0)
    
series

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
1,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
3,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
...,...,...,...,...,...,...,...,...
343,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
344,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
345,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
346,2523-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN


In [54]:
series_clear = series[series.index % 2 != 1]
series_clear

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
6,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
8,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
...,...,...,...,...,...,...,...,...
338,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN
340,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
342,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
344,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN


In [57]:
series_reindex = series_clear.reset_index(drop=True)
series_reindex

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN


In [58]:
series_with_op = series_reindex
series_with_op

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN


In [59]:
import numpy as np

In [61]:
series_with_op['нагрев_operation_1'] = series_with_op['нагрев'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
series_with_op

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3,нагрев_operation_1
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2
1,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2
2,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2
3,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2
4,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3
170,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3
171,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3
172,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3


In [62]:
series_with_op['ковка_operation_2'] = series_with_op['нагрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
series_with_op

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3,нагрев_operation_1,ковка_operation_2
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25
1,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25
2,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25
3,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25
4,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25
...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN
170,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN
171,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN
172,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN


In [63]:
series_with_op['прокат_operation_3'] = series_with_op['нагрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
series_with_op

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3,нагрев_operation_1,ковка_operation_2,прокат_operation_3
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,NaN
1,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,NaN
2,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,NaN
3,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,NaN
4,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,NaN
...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,0.25
170,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,0.25
171,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,0.25
172,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,0.25


In [64]:
for i in [4, 7, 10, 13]:
    if i==4:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op['подогрев'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op['подогрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op['подогрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    elif i==7:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op[f'подогрев.1'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op[f'подогрев.1'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op[f'подогрев.1'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    elif i==10:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op[f'подогрев.2'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op[f'подогрев.2'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op[f'подогрев.2'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    elif i==13:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op[f'подогрев.3'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op[f'подогрев.3'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op[f'подогрев.3'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
        
series_with_op

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3,нагрев_operation_1,ковка_operation_2,...,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
0,"34-8993-18№1,2",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,...,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,...,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,...,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,...,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN,2,0.25,...,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3ч+15м прокат,NaN,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
series_with_op_clear = series_with_op.drop(columns=['нагрев', 'подогрев', 'подогрев.1', 'подогрев.2', 'подогрев.3'])
series_with_op_clear

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев_operation_1,ковка_operation_2,прокат_operation_3,нагрев_operation_4,ковка_operation_5,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
0,"34-8993-18№1,2",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
series_with_op_clear.describe()

,"кол. в садке, шт",Температура,нагрев_operation_1,ковка_operation_2,прокат_operation_3,нагрев_operation_4,ковка_operation_5,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
count,174.000000,174.000000,174.000000,123.00,48.00,84.000000,84.00,0.0,66.000000,66.00,0.0,12.000000,12.00,0.0,0.0,0.0,0.0
mean,1.488506,967.413793,3.011494,0.25,0.25,1.083333,0.25,NaN,0.909091,0.25,NaN,0.833333,0.25,NaN,NaN,NaN,NaN
std,0.501310,69.711885,1.299126,0.00,0.00,0.605364,0.00,NaN,0.289683,0.00,NaN,0.389249,0.00,NaN,NaN,NaN,NaN
min,1.000000,750.000000,1.000000,0.25,0.25,0.000000,0.25,NaN,0.000000,0.25,NaN,0.000000,0.25,NaN,NaN,NaN,NaN
25%,1.000000,950.000000,2.000000,0.25,0.25,1.000000,0.25,NaN,1.000000,0.25,NaN,1.000000,0.25,NaN,NaN,NaN,NaN
50%,1.000000,955.000000,3.000000,0.25,0.25,1.000000,0.25,NaN,1.000000,0.25,NaN,1.000000,0.25,NaN,NaN,NaN,NaN
75%,2.000000,960.000000,3.000000,0.25,0.25,1.000000,0.25,NaN,1.000000,0.25,NaN,1.000000,0.25,NaN,NaN,NaN,NaN
max,2.000000,1100.000000,6.000000,0.25,0.25,4.000000,0.25,NaN,1.000000,0.25,NaN,1.000000,0.25,NaN,NaN,NaN,NaN


In [70]:
series_with_op_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №слитка-индекс-часть  174 non-null    object 
 1   кол. в садке, шт      174 non-null    float64
 2   Температура           174 non-null    float64
 3   нагрев_operation_1    174 non-null    int64  
 4   ковка_operation_2     123 non-null    float64
 5   прокат_operation_3    48 non-null     float64
 6   нагрев_operation_4    84 non-null     float64
 7   ковка_operation_5     84 non-null     float64
 8   прокат_operation_6    0 non-null      float64
 9   нагрев_operation_7    66 non-null     float64
 10  ковка_operation_8     66 non-null     float64
 11  прокат_operation_9    0 non-null      float64
 12  нагрев_operation_10   12 non-null     float64
 13  ковка_operation_11    12 non-null     float64
 14  прокат_operation_12   0 non-null      float64
 15  нагрев_operation_13   0

In [72]:
series_with_op_clear['нагрев_operation_1'] = series_with_op_clear['нагрев_operation_1'].astype(float)
series_with_op_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №слитка-индекс-часть  174 non-null    object 
 1   кол. в садке, шт      174 non-null    float64
 2   Температура           174 non-null    float64
 3   нагрев_operation_1    174 non-null    float64
 4   ковка_operation_2     123 non-null    float64
 5   прокат_operation_3    48 non-null     float64
 6   нагрев_operation_4    84 non-null     float64
 7   ковка_operation_5     84 non-null     float64
 8   прокат_operation_6    0 non-null      float64
 9   нагрев_operation_7    66 non-null     float64
 10  ковка_operation_8     66 non-null     float64
 11  прокат_operation_9    0 non-null      float64
 12  нагрев_operation_10   12 non-null     float64
 13  ковка_operation_11    12 non-null     float64
 14  прокат_operation_12   0 non-null      float64
 15  нагрев_operation_13   0

In [74]:
series_with_op_clear['№слитка-индекс-часть'] = series_with_op_clear['№слитка-индекс-часть'].astype('string')
series_with_op_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №слитка-индекс-часть  174 non-null    string 
 1   кол. в садке, шт      174 non-null    float64
 2   Температура           174 non-null    float64
 3   нагрев_operation_1    174 non-null    float64
 4   ковка_operation_2     123 non-null    float64
 5   прокат_operation_3    48 non-null     float64
 6   нагрев_operation_4    84 non-null     float64
 7   ковка_operation_5     84 non-null     float64
 8   прокат_operation_6    0 non-null      float64
 9   нагрев_operation_7    66 non-null     float64
 10  ковка_operation_8     66 non-null     float64
 11  прокат_operation_9    0 non-null      float64
 12  нагрев_operation_10   12 non-null     float64
 13  ковка_operation_11    12 non-null     float64
 14  прокат_operation_12   0 non-null      float64
 15  нагрев_operation_13   0

In [75]:
series_done = series_with_op_clear
series_done

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев_operation_1,ковка_operation_2,прокат_operation_3,нагрев_operation_4,ковка_operation_5,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
0,"34-8993-18№1,2",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
series_done.to_csv('series_done.csv')

In [186]:
test_df = pd.read_csv('series_done.csv', index_col=0)
test_df

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев_operation_1,ковка_operation_2,прокат_operation_3,нагрев_operation_4,ковка_operation_5,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
0,"34-8993-18№1,2",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
test_df = pd.read_csv('ovens_done.csv', index_col=0)
test_df

,name,ковка,отжиг,прокат,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1


In [76]:
blank = pd.read_excel('ПланОМД.xlsm', sheet_name='Бланк')
blank

,Бланк ОМД,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,Печь,Т-ры,Опер.,от Ø,Нач. т-ра,Посл. т-ра,23-0ч.,NaN,NaN,NaN,...,NaN,21-22ч.,NaN,NaN,NaN,22-23ч.,NaN,NaN,NaN,23-0ч.
1,Печь №2,"960, 950, 830, 860, 1060",ковка,NaN,1060,NaN,"3AL-2,5V, 15143-18",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,[440-385]Б.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1060'C 1шт.,NaN,NaN,NaN,...,NaN,NaN,K,K,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Печь №3,"960, 950, 860, 820, 1000",ковка,NaN,960,NaN,"Э110, 714-18-1слиток",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,[440-390]Б.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,960'C 1шт.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,K,NaN,NaN,NaN


In [86]:
type(blank.iloc[2][0])

float

In [90]:
float('nan')

nan

In [161]:
blank_non_NaN = blank.dropna(how='all')[['Бланк ОМД', 'Unnamed: 1', 'Unnamed: 2']]
blank_non_NaN

,Бланк ОМД,Unnamed: 1,Unnamed: 2
0,Печь,Т-ры,Опер.
1,Печь №2,"960, 950, 830, 860, 1060",ковка
2,NaN,NaN,NaN
3,NaN,NaN,NaN
7,Печь №3,"960, 950, 860, 820, 1000",ковка
8,NaN,NaN,NaN
9,NaN,NaN,NaN
13,Печь №10,"960, 950, 840, 870",ковка
14,NaN,NaN,NaN
15,NaN,NaN,NaN


In [162]:
# in [1, 7, 13, 19, 25, 31, 37]
blank_non_NaN = blank_non_NaN[((blank_non_NaN.index-1)%6==0)&(blank_non_NaN.index<=37)]
blank_non_NaN

,Бланк ОМД,Unnamed: 1,Unnamed: 2
1,Печь №2,"960, 950, 830, 860, 1060",ковка
7,Печь №3,"960, 950, 860, 820, 1000",ковка
13,Печь №10,"960, 950, 840, 870",ковка
19,Печь №15,"960, 950, 840, 870",ковка
25,Печь №20,"1060, 980, 940, 960, 750","прокат, отжиг"
31,Печь №21,"1100, 1060, 960, 950","ковка, прокат"
37,Печь №22,"1100, 1060, 980, 960, 950","ковка, прокат"


In [163]:
blank_reset_index = blank_non_NaN.reset_index(drop=True)
blank_reset_index

,Бланк ОМД,Unnamed: 1,Unnamed: 2
0,Печь №2,"960, 950, 830, 860, 1060",ковка
1,Печь №3,"960, 950, 860, 820, 1000",ковка
2,Печь №10,"960, 950, 840, 870",ковка
3,Печь №15,"960, 950, 840, 870",ковка
4,Печь №20,"1060, 980, 940, 960, 750","прокат, отжиг"
5,Печь №21,"1100, 1060, 960, 950","ковка, прокат"
6,Печь №22,"1100, 1060, 980, 960, 950","ковка, прокат"


In [164]:
blank_reset_index.columns=['name', 'temp', 'operation']
blank_reset_index

,name,temp,operation
0,Печь №2,"960, 950, 830, 860, 1060",ковка
1,Печь №3,"960, 950, 860, 820, 1000",ковка
2,Печь №10,"960, 950, 840, 870",ковка
3,Печь №15,"960, 950, 840, 870",ковка
4,Печь №20,"1060, 980, 940, 960, 750","прокат, отжиг"
5,Печь №21,"1100, 1060, 960, 950","ковка, прокат"
6,Печь №22,"1100, 1060, 980, 960, 950","ковка, прокат"


In [165]:
blank_reset_index[['temp', 'operation']]

,temp,operation
0,"960, 950, 830, 860, 1060",ковка
1,"960, 950, 860, 820, 1000",ковка
2,"960, 950, 840, 870",ковка
3,"960, 950, 840, 870",ковка
4,"1060, 980, 940, 960, 750","прокат, отжиг"
5,"1100, 1060, 960, 950","ковка, прокат"
6,"1100, 1060, 980, 960, 950","ковка, прокат"


In [166]:
def foo1(x):
    return list(x.replace(',', '').split())
blank_reset_index['temp'] = blank_reset_index['temp'].apply(foo1)
blank_reset_index['operation'] = blank_reset_index['operation'].apply(foo1)
# list('1, 2, 3'.replace(',', '').split())
blank_reset_index

,name,temp,operation
0,Печь №2,"[960, 950, 830, 860, 1060]",[ковка]
1,Печь №3,"[960, 950, 860, 820, 1000]",[ковка]
2,Печь №10,"[960, 950, 840, 870]",[ковка]
3,Печь №15,"[960, 950, 840, 870]",[ковка]
4,Печь №20,"[1060, 980, 940, 960, 750]","[прокат, отжиг]"
5,Печь №21,"[1100, 1060, 960, 950]","[ковка, прокат]"
6,Печь №22,"[1100, 1060, 980, 960, 950]","[ковка, прокат]"


In [123]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(blank_reset_index[])
enc.transform(blank_reset_index[['temp', 'operation']]).toarray()

array([[0., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0.]])

In [139]:
# enc.categories_
for row in ['temp', 'operation']:
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(blank_reset_index[[row]])
    blank_reset_index[enc.categories_[0]] = enc.transform(blank_reset_index[[row]]).toarray()

blank_reset_index

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['list']

In [151]:
blank_reset_index

,name,operation
0,Печь №2,[ковка]
1,Печь №3,[ковка]
2,Печь №10,[ковка]
3,Печь №15,[ковка]
4,Печь №20,"[прокат, отжиг]"
5,Печь №21,"[ковка, прокат]"
6,Печь №22,"[ковка, прокат]"


In [160]:
df

,name,ковка,отжиг,прокат
0,Печь №2,1,0,0
1,Печь №3,1,0,0
2,Печь №10,1,0,0
3,Печь №15,1,0,0
4,Печь №20,0,1,1
5,Печь №21,1,0,1
6,Печь №22,1,0,1


In [167]:
df1 = df.copy()

In [170]:
df2 = df.copy()
df2

,name,operation,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,[ковка],0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,[ковка],1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,[ковка],0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,[ковка],0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,"[прокат, отжиг]",0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,"[ковка, прокат]",0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,"[ковка, прокат]",0,1,1,0,0,0,0,0,0,0,1,1,1


In [172]:
df = pd.merge([df1, df2[['name','1000','1060','1100','750','820','830','840','860','870','940','950','960','980']]], left_on='name', right_on='name')
df

TypeError: merge() missing 1 required positional argument: 'right'

In [178]:
pd.concat([df1, df2[['1000','1060','1100','750','820','830','840','860','870','940','950','960','980']]])
df_res = df1.merge(df2, left_on='name', right_on='name', how='inner')
df_res = df_res.drop(columns=['operation'])
df_res

,name,ковка,отжиг,прокат,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1


In [179]:
df_res.to_csv('ovens_done.csv')

In [169]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)
df = blank_reset_index
df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('temp')),
                index=df.index,
                columns=mlb.classes_))
# print(mlb.classes_)
df

,name,operation,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,[ковка],0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,[ковка],1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,[ковка],0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,[ковка],0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,"[прокат, отжиг]",0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,"[ковка, прокат]",0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,"[ковка, прокат]",0,1,1,0,0,0,0,0,0,0,1,1,1


In [127]:
df = blank_reset_index
from sklearn.compose import make_column_transformer
transformer = make_column_transformer(
    (OneHotEncoder(), ['temp', 'operation']),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
print(transformed_df.head())

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [ ]:
blank_reset_index[['temp', 'operation']]

In [105]:

for i in blank.iterrows():
    print(i)
    if i[0] is not float('nan'):
        blank_non_NaN.loc[ len(df.index )] = i
    # df.loc[ len(df.index )] = [value1, value2, value3, ...]
# blank_non_NaN = blank['Печь' in blank.index]
# blank_non_NaN
blank_non_NaN

(0, Бланк ОМД            Печь
Unnamed: 1           Т-ры
Unnamed: 2          Опер.
Unnamed: 3           от Ø
Unnamed: 4      Нач. т-ра
                  ...    
Unnamed: 98       22-23ч.
Unnamed: 99           NaN
Unnamed: 100          NaN
Unnamed: 101          NaN
Unnamed: 102       23-0ч.
Name: 0, Length: 103, dtype: object)


ValueError: cannot set a row with mismatched columns

In [188]:
# снова json
ovens = pd.json_normalize(data['ovens'])
ovens

,start_temp,working_temps,operations
0,1220,"[960, 1030, 1190, 1100, 1230, 1070, 1110, 1080...","[prokat, kovka]"
1,990,"[980, 1190, 1030, 1100, 990]","[otzhig, kovka]"
2,1220,"[970, 1060, 1220]",[otzhig]
3,1020,"[1060, 1100, 1140, 1240, 1150, 1020]",[kovka]
4,1020,"[970, 1100, 1070, 980, 1180, 990, 1020]",[otzhig]
...,...,...,...
417,1240,"[960, 1040, 1180, 1070, 1240]",[kovka]
418,1050,"[1030, 1160, 1040, 980, 1110, 1020, 990, 1050]",[prokat]
419,1140,"[1060, 1230, 1040, 1170, 990, 1140]",[kovka]
420,950,"[1030, 1190, 1160, 1070, 980, 950]",[prokat]


In [189]:
series = pd.json_normalize(data['series'])
series

,temperature,operations
0,1080,"[{'name': 'nagrev', 'timing': 245}, {'name': '..."
1,970,"[{'name': 'nagrev', 'timing': 62}, {'name': 'o..."
2,1080,"[{'name': 'nagrev', 'timing': 76}, {'name': 'k..."
3,980,"[{'name': 'nagrev', 'timing': 96}, {'name': 'o..."
4,980,"[{'name': 'nagrev', 'timing': 82}, {'name': 'p..."
...,...,...
2589,960,"[{'name': 'nagrev', 'timing': 281}, {'name': '..."
2590,1060,"[{'name': 'nagrev', 'timing': 248}, {'name': '..."
2591,1000,"[{'name': 'nagrev', 'timing': 182}, {'name': '..."
2592,1090,"[{'name': 'nagrev', 'timing': 204}, {'name': '..."


In [191]:
def foo3(op_list):
    return pd.DataFrame(op_list)

foo3(series.iloc[0, 1])

,name,timing
0,nagrev,245
1,prokat,15


In [ ]:
mlb = MultiLabelBinarizer(sparse_output=True)
df = blank_reset_index
df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('temp')),
                index=df.index,
                columns=mlb.classes_))
# print(mlb.classes_)
df